In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pickle
import math
from datetime import timezone, datetime
import json

from tqdm import tqdm

import joblib
import matplotlib.pyplot as plt
import mplfinance as mpf
import numba
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
import seaborn as sns
import talib

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score, KFold, TimeSeriesSplit

import optuna
import optuna.integration.lightgbm as lgb_optuna
import lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation

from dtreeviz import trees
from dtreeviz.models.lightgbm_decision_tree import ShadowLightGBMTree

from optuna_util import *
from cpcv_util import *

In [2]:
from timescaledb_util import *

# TimescaleDB用のユーティリティライブラリの設定
_pg_config = {
        'user': os.environ['POSTGRES_USER'],
        'password': os.environ['POSTGRES_PASSWORD'],
        'host': os.environ['POSTGRES_HOST'],
        'port': os.environ['POSTGRES_PORT'],
        'database': os.environ['POSTGRES_DATABASE']
    }

# TimescaleDBアクセス用のユーティリティオブジェクトを作成
_dbutil = TimeScaleDBUtil(user = _pg_config['user'], password = _pg_config['password'], host = _pg_config['host'], port = _pg_config['port'], database = _pg_config['database'])

# 全区間のドルバーの読み込みと保存 (mplfinanceでの表示のためにカラム名を変更している)
df = _dbutil.load_dollarbars('ftx', 'BTC-PERP', 5000000, '2021-01-01 00:00:00.000+00', '2023-01-01 00:00:00.000+00')
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'dollar_volume', 'dollar_buy_volume', 'dollar_sell_volume', 'dollar_liquidation_buy_volume', 'dollar_liquidation_sell_volume', 'dollar_cumsum', 'dollar_buy_cumsum', 'dollar_sell_cumsum']
df['fee'] = 0.02 / 100
df.to_pickle('df_ohlcv.pkl')

# 保存しておいたドルバーを読み込み
#df = pd.read_pickle('df_ohlcv.pkl')
display(df)

,Datetime,Open,High,Low,Close,dollar_volume,dollar_buy_volume,dollar_sell_volume,dollar_liquidation_buy_volume,dollar_liquidation_sell_volume,dollar_cumsum,dollar_buy_cumsum,dollar_sell_cumsum,fee
0,2021-01-01 00:00:36.894000+00:00,29000.0,29029.5,28942.0,28986.5,4.959114e+06,2.564786e+06,2.394328e+06,15531.15400,0.0000,1.923799e+11,9.610236e+10,9.627756e+10,0.0002
1,2021-01-01 00:02:41.379000+00:00,28986.5,29056.0,28979.5,29024.0,5.079592e+06,3.454632e+06,1.624960e+06,206.27575,9175.8848,1.923850e+11,9.610582e+10,9.627918e+10,0.0002
2,2021-01-01 00:08:30.311000+00:00,29024.0,29035.5,28920.0,28942.0,4.998602e+06,8.668237e+05,4.131778e+06,0.00000,0.0000,1.923900e+11,9.610668e+10,9.628331e+10,0.0002
3,2021-01-01 00:10:12.622000+00:00,28942.0,28961.5,28876.0,28907.0,4.991658e+06,2.499264e+06,2.492394e+06,0.00000,0.0000,1.923950e+11,9.610918e+10,9.628581e+10,0.0002
4,2021-01-01 00:13:09.482000+00:00,28907.0,28911.0,28815.5,28850.0,4.896911e+06,1.380208e+06,3.516703e+06,0.00000,6309.5344,1.923999e+11,9.611056e+10,9.628932e+10,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289625,2022-02-10 14:08:30.175000+00:00,43343.0,43382.0,43287.0,43290.0,4.870829e+06,2.014914e+06,2.855915e+06,0.00000,869601.0551,1.641145e+12,8.135513e+11,8.275937e+11,0.0002
289626,2022-02-10 14:09:05.269000+00:00,43282.0,43368.0,43262.0,43328.0,4.995557e+06,3.379773e+06,1.615784e+06,0.00000,0.0000,1.641150e+12,8.135547e+11,8.275953e+11,0.0002
289627,2022-02-10 14:09:32.103000+00:00,43328.0,43400.0,43316.0,43393.0,5.018838e+06,2.991151e+06,2.027687e+06,0.00000,0.0000,1.641155e+12,8.135577e+11,8.275973e+11,0.0002
289628,2022-02-10 14:09:45.133000+00:00,43395.0,43419.0,43372.0,43397.0,4.980746e+06,2.915359e+06,2.065386e+06,0.00000,0.0000,1.641160e+12,8.135606e+11,8.275994e+11,0.0002


In [3]:
# Richmanbtcさんのチュートリアルから拝借した関数群
@numba.njit
def calc_force_entry_price(entry_price=None, lo=None, pips=None):
    y = entry_price.copy()
    y[:] = np.nan
    force_entry_time = entry_price.copy()
    force_entry_time[:] = np.nan
    for i in range(entry_price.size):
        for j in range(i + 1, entry_price.size):
            if round(lo[j] / pips) < round(entry_price[j - 1] / pips):
                y[i] = entry_price[j - 1]
                force_entry_time[i] = j - i
                break
    return y, force_entry_time

def calc_buysell(df, pips = 1, atr_mux = 0.5, atr_range=14):
    # ATRで指値距離を計算します
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=atr_range)
    limit_price_dist = df['ATR'] * atr_mux
    limit_price_dist = np.maximum(1, (limit_price_dist / pips).round().fillna(1)) * pips
    
    # 終値から両側にlimit_price_distだけ離れたところに、買い指値と売り指値を出します
    df['buy_price'] = df['Close'] - limit_price_dist
    df['sell_price'] = df['Close'] + limit_price_dist
    
    # Force Entry Priceの計算
    df['buy_fep'], df['buy_fet'] = calc_force_entry_price(
        entry_price=df['buy_price'].values,
        lo=df['Low'].values,
        pips=pips,
    )

    # calc_force_entry_priceは入力と出力をマイナスにすれば売りに使えます
    df['sell_fep'], df['sell_fet'] = calc_force_entry_price(
        entry_price=-df['sell_price'].values,
        lo=-df['High'].values, # 売りのときは高値
        pips=pips,
    )
    df['sell_fep'] *= -1
    
    horizon = 1 # エントリーしてからエグジットを始めるまでの待ち時間 (1以上である必要がある)
    fee = df['fee'] # maker手数料
    
    # 指値が約定したかどうか (0, 1)
    df['buy_executed'] = ((df['buy_price'] / pips).round() > (df['Low'].shift(-1) / pips).round()).astype('float64')
    df['sell_executed'] = ((df['sell_price'] / pips).round() < (df['High'].shift(-1) / pips).round()).astype('float64')
    
    # yを計算
    df['y_buy'] = np.where(
        df['buy_executed'],
        df['sell_fep'].shift(-horizon) / df['buy_price'] - 1 - 2 * fee,
        0
    )
    df['y_sell'] = np.where(
        df['sell_executed'],
        -(df['buy_fep'].shift(-horizon) / df['sell_price'] - 1) - 2 * fee,
        0
    )

    # バックテストで利用する取引コストを計算
    df['buy_cost'] = np.where(
        df['buy_executed'],
        df['buy_price'] / df['Close'] - 1 + fee,
        0
    )
    df['sell_cost'] = np.where(
        df['sell_executed'],
        -(df['sell_price'] / df['Close'] - 1) + fee,
        0
    )
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df.dropna()

# p平均法 https://note.com/btcml/n/n0d9575882640
def calc_p_mean(x, n):
    ps = []
    for i in range(n):
        x2 = x[i * x.size // n:(i + 1) * x.size // n]
        if np.std(x2) == 0:
            ps.append(1)
        else:
            t, p = ttest_1samp(x2, 0,  alternative='greater')
            if t > 0:
                ps.append(p)
            else:
                ps.append(1)
    return np.mean(ps)

def calc_p_mean_type1_error_rate(p_mean, n):
    return (p_mean * n) ** n / math.factorial(n)

@numba.njit
def backtest(cl=None, hi=None, lo=None, pips=None, buy_entry=None, sell_entry=None, buy_cost=None, sell_cost=None):
    n = cl.size
    y = cl.copy() * 0.0
    poss = cl.copy() * 0.0
    ret = 0.0
    pos = 0.0
    for i in range(n):
        prev_pos = pos
        
        # exit
        if buy_cost[i]:
            vol = np.maximum(0, -prev_pos)
            ret -= buy_cost[i] * vol
            pos += vol

        if sell_cost[i]:
            vol = np.maximum(0, prev_pos)
            ret -= sell_cost[i] * vol
            pos -= vol

        # entry
        if buy_entry[i] and buy_cost[i]:
            vol = np.minimum(1.0, 1 - prev_pos) * buy_entry[i]
            ret -= buy_cost[i] * vol
            pos += vol

        if sell_entry[i] and sell_cost[i]:
            vol = np.minimum(1.0, prev_pos + 1) * sell_entry[i]
            ret -= sell_cost[i] * vol
            pos -= vol
        
        if i + 1 < n:
            ret += pos * (cl[i + 1] / cl[i] - 1)
            
        y[i] = ret
        poss[i] = pos
        
    return y, poss

In [4]:
# Richmanbtcさんのチュートリアルの関数をカスタムした関数群

def calc_features(df):
    open = df['Open']
    high = df['High']
    low = df['Low']
    close = df['Close']
    volume = df['dollar_buy_volume'] + df['dollar_sell_volume']
        
    # ドルバー由来の特徴量を追加
    df['TIME'] = df.index
    df['DELTA_TIME'] = df['TIME'].diff()
    df['DELTA_TIME'] = df['DELTA_TIME'].view(int) / 1_000_000_000 # 単位は秒
    df['DELTA_TIME_T-1'] = df['DELTA_TIME'].shift(1)
    df['DELTA_TIME_T-2'] = df['DELTA_TIME'].shift(2)
    df['DELTA_TIME_T-3'] = df['DELTA_TIME'].shift(3)
    df['DELTA_TIME_T-4'] = df['DELTA_TIME'].shift(4)
    
    df['DELTA_TIME_T-1/T'] = df['DELTA_TIME_T-1'] / df['DELTA_TIME']
    df['DELTA_TIME_T-2/T'] = df['DELTA_TIME_T-2'] / df['DELTA_TIME']
    df['DELTA_TIME_T-3/T'] = df['DELTA_TIME_T-3'] / df['DELTA_TIME']
    df['DELTA_TIME_T-4/T'] = df['DELTA_TIME_T-4'] / df['DELTA_TIME']
    
    df['BUY_RATIO'] = df['dollar_buy_volume'] / (df['dollar_buy_volume'] + df['dollar_sell_volume'])
    
    df['BUY_RATIO_DIFF_T-1'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(1)
    df['BUY_RATIO_DIFF_T-2'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(2)
    df['BUY_RATIO_DIFF_T-3'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(3)
    df['BUY_RATIO_DIFF_T-4'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(4)

    df['SELL_RATIO'] = df['dollar_sell_volume'] / (df['dollar_buy_volume'] + df['dollar_sell_volume'])

    df['SELL_RATIO_DIFF_T-1'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(1)
    df['SELL_RATIO_DIFF_T-2'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(2)
    df['SELL_RATIO_DIFF_T-3'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(3)
    df['SELL_RATIO_DIFF_T-4'] = df['BUY_RATIO'] - df['BUY_RATIO'].shift(4)

    df['LIQUIDATION_BUY_RATIO'] = df['dollar_liquidation_buy_volume'] / (df['dollar_buy_volume'] + df['dollar_sell_volume'])
    df['LIQUIDATION_BUY_RATIO_DIFF_T-1'] = df['LIQUIDATION_BUY_RATIO'] - df['LIQUIDATION_BUY_RATIO'].shift(1)
    df['LIQUIDATION_BUY_RATIO_DIFF_T-2'] = df['LIQUIDATION_BUY_RATIO'] - df['LIQUIDATION_BUY_RATIO'].shift(2)
    df['LIQUIDATION_BUY_RATIO_DIFF_T-3'] = df['LIQUIDATION_BUY_RATIO'] - df['LIQUIDATION_BUY_RATIO'].shift(3)
    df['LIQUIDATION_BUY_RATIO_DIFF_T-4'] = df['LIQUIDATION_BUY_RATIO'] - df['LIQUIDATION_BUY_RATIO'].shift(4)

    df['LIQUIDATION_SELL_RATIO'] = df['dollar_liquidation_sell_volume'] / (df['dollar_buy_volume'] + df['dollar_sell_volume'])
    df['LIQUIDATION_SELL_RATIO_DIFF_T-1'] = df['LIQUIDATION_SELL_RATIO'] - df['LIQUIDATION_SELL_RATIO'].shift(1)
    df['LIQUIDATION_SELL_RATIO_DIFF_T-2'] = df['LIQUIDATION_SELL_RATIO'] - df['LIQUIDATION_SELL_RATIO'].shift(2)
    df['LIQUIDATION_SELL_RATIO_DIFF_T-3'] = df['LIQUIDATION_SELL_RATIO'] - df['LIQUIDATION_SELL_RATIO'].shift(3)
    df['LIQUIDATION_SELL_RATIO_DIFF_T-4'] = df['LIQUIDATION_SELL_RATIO'] - df['LIQUIDATION_SELL_RATIO'].shift(4)
    
    return df.dropna()

# 損切、利確指値と高値、安値の系列から、損切と利確が行われるポイントを探索する関数
#@numba.njit # デバッグprintがあるので今はnumbaの高速化を使わないでおく
def calc_ptsl_price(executed=None, horizon=None, entry_price=None, pt_price=None, sl_price=None, low=None, high=None, pips=None, debug=False):
    y = entry_price.copy()
    y[:] = np.nan
    exit_time = entry_price.copy()
    exit_time[:] = np.nan
    for i in range(entry_price.size):
        # インデックスiの次のバーでエントリー指値が刺さっていなかったら、利確損切の結果を探索しない
        if executed[i] == 0:
            y[i] = entry_price[i]
            exit_time[i] = 0
            continue
        
        for j in range(i + 1 + horizon, entry_price.size):
            if debug is True:
                print(f'i {i}, j {j}, entry {entry_price[i]}, pt {pt_price[i]}, sl {sl_price[i]}, low {low[j]}, high {high[j]}')
            if round(low[j] / pips) < round(pt_price[i] / pips) and round(high[j] / pips) > round(pt_price[i] / pips):
                y[i] = pt_price[i]
                exit_time[i] = j - i
                if debug is True:
                    print(f'take profit')
                break
            if round(low[j] / pips) < round(sl_price[i] / pips) and round(high[j] / pips) > round(sl_price[i] / pips):
                y[i] = sl_price[i]
                exit_time[i] = j - i
                if debug is True:
                    print(f'stop loss')
                break
    return y, exit_time

# 損切り利確がある執行を模擬する関数
def calc_buysell_ptsl(df, pips = 1, atr_mux = 0.5, atr_range=14):
    # ATRで指値距離を計算します
    df.loc[:, 'ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=atr_range)
    df.dropna(inplace=True)
    limit_price_dist = df['ATR'] * atr_mux
    limit_price_dist = np.maximum(1, (limit_price_dist / pips).round().fillna(1)) * pips
    
    # 終値から両側にlimit_price_distだけ離れたところに、エントリー指値を出します
    # さらに、買い指値と売り指値から両側に0.2%だけ離れたところに、利確指値と損切指値を出します
    df['buy_price'] = df['Close'] - limit_price_dist    
    df['buy_pt_price'] = df['buy_price'] + df['buy_price'] * 0.2 / 100
    df['buy_sl_price'] = df['buy_price'] - df['buy_price'] * 0.2 / 100
    df['sell_price'] = df['Close'] + limit_price_dist
    df['sell_pt_price'] = df['sell_price'] - df['sell_price'] * 0.2 / 100
    df['sell_sl_price'] = df['sell_price'] + df['sell_price'] * 0.2 / 100
    
    # エントリー指値が約定したかどうかをデータフレームに追加します (0.0か1.0)
    df['buy_executed'] = ((df['buy_price'] / pips).round() > (df['Low'].shift(-1) / pips).round()).astype('float64')
    df['sell_executed'] = ((df['sell_price'] / pips).round() < (df['High'].shift(-1) / pips).round()).astype('float64')
    
    horizon = 1 # エントリーしてからエグジットを始めるまでの待ち時間 (1以上でないと、一本の足の中で常に有利な購入即売却をしたとみなすので1以上である必要があります)
    fee = df['fee'] # maker手数料

    # 買いエグジット価格の計算
    df['buy_exit_price'], df['buy_fet'] = calc_ptsl_price(
        executed = df['buy_executed'].values,
        horizon = horizon,
        entry_price = df['buy_price'].values,
        pt_price = df['buy_pt_price'].values,
        sl_price = df['buy_sl_price'].values,
        low = df['Low'].values,
        high = df['High'].values,
        pips=pips,
        debug=False
    )

    # calc_ptsl_priceも、calc_force_entry_priceと同様に入力と出力をマイナスにすれば売りエグジット価格の計算に使えます
    df['sell_exit_price'], df['sell_fet'] = calc_ptsl_price(
        executed = df['sell_executed'].values,
        horizon = horizon,
        entry_price = -df['sell_price'].values,
        pt_price = -df['sell_pt_price'].values,
        sl_price = -df['sell_sl_price'].values,
        low = -df['High'].values,
        high = -df['Low'].values,
        pips=pips,
        debug = False
    )
    df['sell_exit_price'] *= -1
            
    # yを計算
    df['y_buy'] = np.where(
        df['buy_executed'],
        df['buy_exit_price'] / df['buy_price'] - 1 - 2 * fee,
        0
    )
    df['y_sell'] = np.where(
        df['sell_executed'],
        -(df['sell_exit_price'] / df['sell_price'] - 1) - 2 * fee,
        0
    )

    # バックテストで利用する取引コストを計算
    df['buy_cost'] = np.where(
        df['buy_executed'],
        df['buy_price'] / df['Close'] - 1 + fee,
        0
    )
    df['sell_cost'] = np.where(
        df['sell_executed'],
        -(df['sell_price'] / df['Close'] - 1) + fee,
        0
    )
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df.dropna()

In [5]:
from sklearn.metrics import mean_squared_error

# パフォーマンスを表示するためのユーティリティ関数
def display_performance(df, subtitle):
    fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    plt.subplots_adjust(wspace=0.2, hspace=0.3)
    
    df['buy_executed'].rolling(1000).mean().plot(label='Buy', ax=ax[0, 0])
    df['sell_executed'].rolling(1000).mean().plot(label='Sell', ax=ax[0, 0])
    ax[0, 0].set_title(f'Execution probability ({subtitle})')
    ax[0, 0].legend()

    df['buy_fet'].rolling(1000).mean().plot(label='Buy', ax=ax[0, 1])
    df['sell_fet'].rolling(1000).mean().plot(label='Sell',  ax=ax[0, 1])
    ax[0, 1].set_title(f'Average time until exit ({subtitle})')
    ax[0, 1].legend()

    df['buy_fet'].hist(alpha=0.3, label='Buy', ax=ax[1, 0])
    df['sell_fet'].hist(alpha=0.3, label='Sell', ax=ax[1, 0])
    ax[1, 0].set_title(f'Time distribution until exit ({subtitle})')
    ax[1, 0].legend()

    (df['buy_executed'] * df['y_buy']).cumsum().plot(label='Buy', ax=ax[1, 1])
    (df['sell_executed'] * df['y_sell']).cumsum().plot(label='Sell', ax=ax[1, 1])
    (df['buy_executed'] * df['y_buy'] + df['sell_executed'] * df['y_sell']).cumsum().plot(label='Buy+Sell', ax=ax[1, 1])
    ax[1, 1].set_title(f'Cumulative return ({subtitle})')
    ax[1, 1].legend()
    fig.show()
    
    # 数値を表示する
    buy_return = (df["buy_executed"] * df["y_buy"]).sum()
    sell_return = (df["sell_executed"] * df["y_sell"]).sum()
    
    bar_count = len(df)
    
    buy_count = len(df[(df["buy_executed"] > 0)])
    sell_count = len(df[(df["sell_executed"] > 0)])
    
    positive_buy_count = len(df[(df["buy_executed"] > 0) & (df["y_buy"] > 0)])
    negative_buy_count = len(df[(df["buy_executed"] > 0) & (df["y_buy"] < 0)])
    
    positive_sell_count = len(df[(df["sell_executed"] > 0) & (df["y_sell"] > 0)])
    negative_sell_count = len(df[(df["sell_executed"] > 0) & (df["y_sell"] < 0)])

    positive_buy_return = df.loc[(df["buy_executed"] > 0) & (df["y_buy"] > 0), "y_buy"].sum()
    negative_buy_return = df.loc[(df["buy_executed"] > 0) & (df["y_buy"] < 0), "y_buy"].sum()
                           
    positive_sell_return = df.loc[(df["sell_executed"] > 0) & (df["y_sell"] > 0), "y_sell"].sum()
    negative_sell_return = df.loc[(df["sell_executed"] > 0) & (df["y_sell"] < 0), "y_sell"].sum()

    print(f'バーの数 {bar_count}本')
    print(f'買いリターン合計 {buy_return * 100:.2f}% 売りリターン合計 {sell_return*100:.2f}% 合計リターン {(buy_return + sell_return)*100:.2f}%')
    print(f'買い {buy_count}回, 売り {sell_count}回')
    print(f'リターンが正の買い {positive_buy_count}回 ({(positive_buy_count / buy_count)*100:.02f} %), リターンが負の買い {negative_buy_count}回 ({(negative_buy_count / buy_count)*100:.02f} %)')
    print(f'リターンが正の売り {positive_sell_count}回 ({(positive_sell_count / sell_count)*100:.02f} %), リターンが負の売り {negative_sell_count}回 ({(negative_sell_count / sell_count)*100:.02f} %)')
    print(f'正の買いリターン平均 {positive_buy_return / positive_buy_count * 100:.03f} %, 負の買いリターン平均 {negative_buy_return / negative_buy_count * 100:.03f} %')
    print(f'正の売りリターン平均 {positive_sell_return / positive_sell_count * 100:.03f} %, 負の売りリターン平均 {negative_sell_return / negative_sell_count * 100:.03f} %')
    print(f'買いリターン平均 {(buy_return / buy_count)*100:.03f} %, 売りリターン平均 {(sell_return / sell_count)*100:.03f} %')

# 真値と予測値を比較して表示するためのユーティリティ関数
def display_true_vs_pred(series_true, series_pred, subtitle):
    df = pd.DataFrame({'true': series_true, 'pred': series_pred}).dropna()
    corr = np.corrcoef(df['true'], df['pred'])
    pred_std = df['pred'].std()
    pred_mean = df['pred'].mean()
    true_std = df['true'].std()
    true_mean = df['true'].mean()
    
    pred_max = pred_mean + 4*pred_std
    pred_min = pred_mean - 4*pred_std
    true_max = true_mean + 4*true_std
    true_min = true_mean - 4*true_std
    
    fig, ax = plt.subplots(2, 2, sharex='col', sharey = 'row',
                           gridspec_kw={'width_ratios': [2, 0.5], 'height_ratios': [2, 0.5]}, figsize=(6, 6))
    
    # 予測値のレンジごとに真値の平均をプロットして比例関係があることを確認する
    pred_sections = []
    true_means = []
    for i in range(17):
        _df = df[(df['pred'] >= pred_min + 0.5 * pred_std * i) & (df['pred'] < pred_min + 0.5 * pred_std * (i + 1))]
        pred_sections.append(pred_min + 0.5 * pred_std * i)
        true_means.append(_df['true'].mean())

    # 散布図をプロットする (予測対象にならなかったtrue値は0になっているのでそれを除外していることに注意)
    #ax[0, 0].scatter(df.loc[df['true'] != 0,'pred'], df.loc[df['true'] != 0, 'true'], s = 1)
    ax[0, 0].scatter(df['pred'], df['true'], s = 1)
    ax[0, 0].step(pred_sections, true_means, 'red', where='post')
    ax[0, 0].set_title(subtitle + f' (IC={corr[0][1]:0.4f})')
    ax[0, 0].set_xlabel("prediction")
    ax[0, 0].set_ylabel("truth")
    ax[0, 0].set_xlim([pred_min, pred_max])
    ax[0, 0].set_ylim([true_min, true_max])
    ax[0, 0].set_xticks([pred_mean, pred_mean - 2*pred_std, pred_mean - 4*pred_std, pred_mean + 2*pred_std, pred_mean + 4*pred_std])
    ax[0, 0].set_yticks([true_mean, true_mean - 2*true_std, true_mean - 4*true_std, true_mean + 2*true_std, true_mean + 4*true_std])
    ax[0, 0].grid(axis = 'both')
    ax[0, 0].axvline(0, color='red', linestyle='dotted', linewidth=1)
    ax[0, 0].axhline(0, color='red', linestyle='dotted', linewidth=1)
    
    # 0と予測した値を表示するとヒストグラムがまともに見えないので除外している
    ax[1, 0].hist(df.loc[df['true'] != 0, 'pred'], bins=50, range=[pred_min, pred_max])
    ax[1, 0].grid(axis = 'both')
    ax[1, 0].axvline(0, color='red', linestyle='dotted', linewidth=1)
    
    ax[0, 1].hist(df.loc[df['true'] != 0, 'true'], bins=50, orientation='horizontal', range=[true_min, true_max])
    ax[0, 1].grid(axis = 'both')
    ax[0, 1].axhline(0, color='red', linestyle='dotted', linewidth=1)
    
    ax[1, 1].remove()
    
    fig.show()

# バックテスト結果を表示するためのユーティリティ関数
def display_backtest(df):
    # バックテストで累積リターンと、ポジションを計算
    df['cum_ret'], df['poss'] = backtest(
        cl = df['Close'].values,
        buy_entry = df['y_pred_buy'].values > 0,
        sell_entry = df['y_pred_sell'].values > 0,
        buy_cost = df['buy_cost'].values,
        sell_cost = df['sell_cost'].values)
    
    fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    plt.subplots_adjust(wspace=0.2, hspace=0.3)

    df['cum_ret'].plot(ax = ax[0, 0])
    ax[0, 0].set_title('Cumulative return')

    df['poss'].plot(ax = ax[0, 1])
    ax[0, 1].set_title('Position history')

    df['poss'].rolling(1000).mean().plot(ax = ax[1, 0])
    ax[1, 0].set_title('Position average history')
    
    df['poss'].diff(1).abs().dropna().cumsum().plot(ax = ax[1, 1])
    ax[1, 1].set_title('Cumulative trade volume')
    fig.show()

    print('t test')
    x = df['cum_ret'].diff(1).dropna()
    t, p = ttest_1samp(x, 0, alternative='greater')
    print('t value {}'.format(t))
    print('p value {}'.format(p))

    x = df['cum_ret'].diff(1).dropna()
    p_mean_n = 5
    p_mean = calc_p_mean(x, p_mean_n)
    print('p平均法 n = {}'.format(p_mean_n))
    print('p平均 {}'.format(p_mean))

In [ ]:
# ATRの指値幅を変数として、取引数とリターンの積を最大化するOptunaの最適化対象の関数
def target_function(x, y):
    df = pd.read_pickle('df_ohlcv.pkl')
    df = calc_buysell(df, pips = 1, atr_mux = x, atr_range=y)
    
    y_buy = (df['buy_executed'] * df['y_buy']).sum()
    y_sell = (df['sell_executed'] * df['y_sell']).sum()
    
    #print(f'y_buy = {y_buy}, y_sell = {y_sell}, atr_mux = {x}, atr_range = {y}')
    return y_buy + y_sell

# Optunaの最適化目的関数
def objective(trial):
    x = trial.suggest_uniform('x', 0.5, 2.0)
    y = trial.suggest_int('y', 1, 21)
    score = target_function(x, y)
    return score

# 最適化を実施
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 2000)
atr_mux = study.best_params['x']
atr_range = study.best_params['y']
optuna.visualization.plot_slice(study, params=["x"])

print(f'Best atr_mux = {atr_mux}, best atr_range = {atr_range}')

# 以前の学習結果を利用して最適化をスキップ
# 10_000_000ドルバー : Trial 74 finished with value: 16193.839450781088 and parameters: {'x': 1.4788760220633528}. Best is trial 74 with value: 16193.839450781088
# 5_000_000ドルバー : Trial 193 finished with value: 7080.370047388982 and parameters: {'x': 2.1472956042094262}. Best is trial 193 with value: 7080.370047388982.
#atr_mux = 1.4788
#atr_range = 4

[I 2022-02-10 14:26:53,717] A new study created in memory with name: no-name-d5f93077-e15c-42bf-8d26-87a79f3a6d20
[I 2022-02-10 14:26:54,251] Trial 0 finished with value: -60.27764754721869 and parameters: {'x': 0.8158284704889938, 'y': 11}. Best is trial 0 with value: -60.27764754721869.
[I 2022-02-10 14:26:54,397] Trial 1 finished with value: -72.57420311100427 and parameters: {'x': 0.7449003578679538, 'y': 19}. Best is trial 0 with value: -60.27764754721869.
[I 2022-02-10 14:26:54,550] Trial 2 finished with value: -42.883443991028585 and parameters: {'x': 0.9315036599094086, 'y': 13}. Best is trial 2 with value: -42.883443991028585.
[I 2022-02-10 14:26:54,890] Trial 3 finished with value: -3.6925856564951087 and parameters: {'x': 1.7829382402156024, 'y': 20}. Best is trial 3 with value: -3.6925856564951087.
[I 2022-02-10 14:26:55,233] Trial 4 finished with value: -2.597409938963515 and parameters: {'x': 1.806798565313214, 'y': 15}. Best is trial 4 with value: -2.597409938963515.
[I 

In [ ]:
# 学習区間でのパフォーマンス (MLフィルタ適応前) を表示
df_atr_optimized = pd.read_pickle('df_ohlcv.pkl')
df_atr_optimized = calc_features(df_atr_optimized)
#df_atr_optimized = calc_buysell_ptsl(df_atr_optimized, pips = 1, atr_mux = atr_mux, atr_range = atr_range)
df_atr_optimized = calc_buysell(df_atr_optimized, pips = 1, atr_mux = atr_mux, atr_range = atr_range)

# すべてのバーで売買しようとした結果を表示したいので、全ての行で正のリターンを予測したことにしておく
df_atr_optimized['y_pred_buy'] = 1.0
df_atr_optimized['y_pred_sell'] = 1.0

display_performance(df_atr_optimized, 'Train, without ML')

In [ ]:
# 学習に利用しない特徴量はコメントアウトすること
features = sorted([
    'BUY_RATIO',
    'BUY_RATIO_DIFF_T-1', 'BUY_RATIO_DIFF_T-2', 'BUY_RATIO_DIFF_T-3', 'BUY_RATIO_DIFF_T-4',
    'SELL_RATIO', 'SELL_RATIO_DIFF_T-1', 'SELL_RATIO_DIFF_T-2', 'SELL_RATIO_DIFF_T-3', 'SELL_RATIO_DIFF_T-4',
    'LIQUIDATION_BUY_RATIO', 'LIQUIDATION_BUY_RATIO_DIFF_T-1', 'LIQUIDATION_BUY_RATIO_DIFF_T-2', 'LIQUIDATION_BUY_RATIO_DIFF_T-3', 'LIQUIDATION_BUY_RATIO_DIFF_T-4',
    'LIQUIDATION_SELL_RATIO', 'LIQUIDATION_SELL_RATIO_DIFF_T-1', 'LIQUIDATION_SELL_RATIO_DIFF_T-2', 'LIQUIDATION_SELL_RATIO_DIFF_T-3', 'LIQUIDATION_SELL_RATIO_DIFF_T-4',
    'DELTA_TIME_T-1/T', 'DELTA_TIME_T-2/T', 'DELTA_TIME_T-3/T', 'DELTA_TIME_T-4/T',
])

In [ ]:
# 保存済みのデータフレームを読み込む
df = pd.read_pickle('df_ohlcv.pkl')

# トレーニングデータに売買アクションと特徴量を追加
df_train = df[(df['Datetime'] >= '2021-01-01 00:00:00+00') & (df['Datetime'] < '2022-01-01 00:00:00+00')].copy()
df_train = calc_features(df_train)
df_train = calc_buysell(df_train, pips = 1, atr_mux = atr_mux, atr_range = atr_range)

buy_lgb_util = LightGBMRegressionUtil()
sell_lgb_util = LightGBMRegressionUtil()

# Optunaを使わないシンプルなregression予測を行い、トレーニング区間を予測する
folds = cpcv_split(df_train)

buy_lgb_util.train_cv(df_train[features], df_train['y_buy'], folds, use_optuna = False)
buy_lgb_util.save_model('y_buy.model.xz', compress = True)

sell_lgb_util.train_cv(df_train[features], df_train['y_sell'], folds, use_optuna = False)
sell_lgb_util.save_model('y_sell.model.xz', compress = True)

In [ ]:
# トレーニング区間のデータに売買アクションと特徴量を追加
df_train = df[(df['Datetime'] >= '2021-01-01 00:00:00+00') & (df['Datetime'] < '2022-01-01 00:00:00+00')].reset_index().copy()
df_train = calc_features(df_train)
#df_train = calc_buysell_ptsl(df_train, pips = 1, atr_mux = atr_mux, atr_range = atr_range)
df_train = calc_buysell(df_train, pips = 1, atr_mux = atr_mux, atr_range = atr_range)

# 学習区間について予測を行い、学習したモデルの性能を確認する
df_train['y_pred_buy'] = buy_lgb_util.predict(df_train[features], folds, predict_training = True)
df_train['y_pred_sell'] = sell_lgb_util.predict(df_train[features], folds, predict_training = True)

# 真値と予測値の関係を見る
display_true_vs_pred(df_train['y_buy'], df_train['y_pred_buy'], 'Prediction vs Truth (BUY model, training section)')
display_true_vs_pred(df_train['y_sell'], df_train['y_pred_sell'], 'Prediction vs Truth (SELL model, training section)')

In [ ]:
# トレーニング区間のデータに売買アクションと特徴量を追加
df_train = df[(df['Datetime'] >= '2021-01-01 00:00:00+00') & (df['Datetime'] < '2022-01-01 00:00:00+00')].reset_index().copy()
df_train = calc_features(df_train)
#df_train = calc_buysell_ptsl(df_train, pips = 1, atr_mux = atr_mux, atr_range = atr_range)
df_train = calc_buysell(df_train, pips = 1, atr_mux = atr_mux, atr_range = atr_range)

# バリデーション区間について予測を行い、学習したモデルの性能を確認する
df_train['y_pred_buy'] = buy_lgb_util.predict(df_train[features], folds)
df_train['y_pred_sell'] = sell_lgb_util.predict(df_train[features], folds)

# 真値と予測値の関係を見る
display_true_vs_pred(df_train['y_buy'], df_train['y_pred_buy'], 'Prediction vs Truth (BUY model, validation section)')
display_true_vs_pred(df_train['y_sell'], df_train['y_pred_sell'], 'Prediction vs Truth (SELL model, validation section)')

In [ ]:
# テスト区間のデータに売買アクションと特徴量を追加
df_test = df[(df['Datetime'] >= '2022-01-01 00:00:00+00') & (df['Datetime'] < '2023-01-01 00:00:00+00')].reset_index().copy()
df_test = calc_features(df_test)
#df_test = calc_buysell_ptsl(df_test, pips = 1, atr_mux = atr_mux, atr_range = atr_range)
df_test = calc_buysell(df_test, pips = 1, atr_mux = atr_mux, atr_range = atr_range)

# 学習区間について予測を行い、学習したモデルの性能を確認する
df_test['y_pred_buy'] = buy_lgb_util.predict(df_test[features])
df_test['y_pred_sell'] = sell_lgb_util.predict(df_test[features])

# 真値と予測値の関係を見る
display_true_vs_pred(df_test['y_buy'], df_test['y_pred_buy'], 'Prediction vs Truth (BUY model, test section)')
display_true_vs_pred(df_test['y_sell'], df_test['y_pred_sell'], 'Prediction vs Truth (SELL model, test section)')

In [ ]:
# 予測が正でないフレームのbuy_executedをすべて0にしておく
#df_test.loc[df_test['y_pred_buy'] <= 0, 'buy_executed'] = 0.0
#df_test.loc[df_test['y_pred_sell'] <= 0, 'sell_executed'] = 0.0

# パフォーマンスの表示
#display_performance(df_test, 'Validation, with ML')
#display_backtest(df_test)

In [ ]:
# CPCVによって生成されたboosterのうち一つを選んで特徴量の重要度を表示する
lgb.plot_importance(model_buy.boosters[0], title='Feature importance (BUY model)', figsize=(6, 6))
lgb.plot_importance(model_sell.boosters[0], title='Feature importance (SELL model)', figsize=(6, 6))

In [ ]:
# dtreevizを利用した決定木の可視化
#light_dtree_reg = ShadowLightGBMTree(model_buy.boosters[0], tree_index = 1, x_data=df_test[features], y_data=df_test['y_buy'], feature_names=features, target_name='y_buy')
#viz = trees.dtreeviz(light_dtree_reg, show_node_labels = True, precision = 6)
#viz.save('./lgb_tree.svg')
#print(f'SVG saved.')